In [151]:
from shapely.geometry import Point
import geopandas as gp

In [152]:
from leaflet_creation import create_map_visualization

In [153]:
raw_gpx_dir = 'Sample Data/'
snapped_gpx_dir = 'Sample Data/split/'
gpx_video_match_file = 'Sample Data/gpx_video_match.csv'

In [154]:
import pandas as pd
gpx_video_match = pd.read_csv(gpx_video_match_file,index_col=0)
gpx_video_match['snapped'] = gpx_video_match.vfile.apply(lambda x: snapped_gpx_dir + x.replace('.MP4', '.json'))
gpx_video_match['gpx'] = gpx_video_match.gpx.apply(lambda x: raw_gpx_dir + x)
gpx_video_match.drop('vfile', inplace=True, axis=1)

In [155]:



def conf2color(conf):
    if conf>0.8:
        return '#1CFF00'
    if conf>0.5:
        return '#4EFFC5'
    if conf>0.1:
        return '#E8AE19'
    return '#FF100F'

In [156]:
from utils import find_files

In [157]:
raw_list = [x.replace('\\','/') for x in find_files(raw_gpx_dir, '*.gpx')]
df_raw = pd.DataFrame(raw_list, columns=['gpx'])

In [158]:
snapped_list = [x.replace('\\','/') for x in find_files(snapped_gpx_dir, '*.json')]
df_snapped = pd.DataFrame(snapped_list, columns=['snapped'])


In [159]:
from split_video_gpx import parse_gpx

In [174]:
Point(1,2).coords[0]

(1.0, 2.0)

In [187]:
bfr = 0.000008
gpdfs = []
binding_data = []
for i, raw_gpx in enumerate(raw_list):
    vfile, locs, tms = parse_gpx(raw_gpx)
    snapped = os.path.join(snapped_gpx_dir, vfile).replace('.MP4', '.json')
    
    if not os.path.isfile(snapped):
        continue
    print i, raw_gpx, snapped
    
    gpdf_raw = gp.GeoDataFrame(zip([Point(x) for x in locs], tms), columns=['geometry', 'timestamp']).reset_index()
    gpdf_raw['lat_lon'] = gpdf_raw.geometry.apply(lambda x: (x.coords[0][1], x.coords[0][0]))
    gpdf_raw['color'] = '#aaa'
    gpdf_raw.geometry = gpdf_raw.buffer(bfr)
    gpdfs.append(gpdf_raw)
    binding_data.append(['ly%d'%(2*i), 'raw%d'%i])
    
    snapped_json = load_json_file(snapped)
    gpdf_snapped = []
    for subv in snapped_json:
        vname = subv['video_name']
        confidences = subv['confidences']
        locs = subv['lonlat']
        if len(confidences)>1:
            print vname, len(confidences)
        for batch in zip(confidences,locs):
            raw = batch[0]['#origin_pts']
            snap = batch[0]['#snap_pts']
            batch_id = batch[0]['#batch']
            conf = batch[0]['confidence']
            color = conf2color(conf)
            batch_locs = batch[1]
            gpdf_snapped.extend([(vname, raw, snap, batch_id, conf,color, 
                                  Point(loc).buffer((1+int(batch_id)*0.5)*bfr),
                                  (loc[1],loc[0])) for loc in batch_locs])
    gpdf_snapped = gp.GeoDataFrame(gpdf_snapped, columns=['vname', '#raw_points', '#snapped_points', 
                                                          'batch_id','confidence', 'color', 'geometry', 'latlon']).reset_index()
    gpdfs.append(gpdf_snapped)
    binding_data.append(['ly%d'%(2*i+1), 'snapped%i'%i])


3 Sample Data/GPX/Track_2017-02-21 110332.gpx Sample Data/split/DCIM/103_VIRB/VIRB0006.json
4 Sample Data/GPX/Track_2017-02-21 110604.gpx Sample Data/split/DCIM/103_VIRB/VIRB0007.json
5 Sample Data/GPX/Track_2017-02-21 113002.gpx Sample Data/split/DCIM/103_VIRB/VIRB0008.json
6 Sample Data/GPX/Track_2017-02-21 114737.gpx Sample Data/split/DCIM/103_VIRB/VIRB0009.json
7 Sample Data/GPX/Track_2017-02-21 120031.gpx Sample Data/split/DCIM/103_VIRB/VIRB0010.json
10 Sample Data/GPX/Track_2017-02-21 125536.gpx Sample Data/split/DCIM/104_VIRB/VIRB0002.json
11 Sample Data/GPX/Track_2017-02-21 131933.gpx Sample Data/split/DCIM/104_VIRB/VIRB0003.json
split/DCIM/104_VIRB/VIRB0003_000.MP4 2
split/DCIM/104_VIRB/VIRB0003_016.MP4 2
12 Sample Data/GPX/Track_2017-02-21 134158.gpx Sample Data/split/DCIM/104_VIRB/VIRB0004.json


In [188]:
html_title = 'SNAP'
file_path = ''
file_name = 'SNAP'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [166]:
from split_video_gpx import second2vtime
second2vtime(823)

'00:13:43'

In [178]:
from trace2seg import snap2road

In [191]:
gpx = 'Sample Data/GPX/Track_2017-02-21 125536.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly1'
file_path = ''
file_name = 'anomaly1'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)

In [193]:
gpx = 'Sample Data/GPX/Track_2017-02-21 131933.gpx'
vfile, raw_locs, tms = parse_gpx(gpx)

snapped_locs = snap2road(raw_locs)

gpdf_raw = gp.GeoDataFrame([Point(x) for x in raw_locs], columns=['geometry']).reset_index()
gpdf_raw.geometry = gpdf_raw.buffer(bfr)
gpdf_raw['color'] = '#aaa'
gpdf_snapped = gp.GeoDataFrame([Point(x) for x in snapped_locs], columns=['geometry']).reset_index()
gpdf_snapped.geometry = gpdf_snapped.buffer(bfr)
gpdf_snapped['color'] = '#800'
gpdfs = [gpdf_raw, gpdf_snapped]
binding_data = [['ly1', 'raw'], ['ly2','snapped']]

html_title = 'anomaly2'
file_path = ''
file_name = 'anomaly2'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets']
map_layers = ['light','streets', 'satellite']
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs)